In [1]:
import torch.nn as nn
from transformers import DistilBertModel, DistilBertTokenizer
from transformers import Trainer, TrainingArguments
import torch
import pandas as pd
import os

/home/eaglewatch/miniconda3/envs/kobertnlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-28 10:59:13.070994: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-28 10:59:13.231637: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 10:59:13.231663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-2

In [3]:
tokenizer_path = "distilbert-base-multilingual-cased"
tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_path)


In [4]:
class DistilBertClassifier(nn.Module):
    def __init__(self, distilbert_type="distilbert-base-multilingual-cased", num_labels=2):
        super(DistilBertClassifier, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained(distilbert_type)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.distilbert.config.dim, num_labels)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        # Using the [CLS] token's representation (first token)
        pooled_output = outputs[0][:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [5]:
model = DistilBertClassifier(num_labels=775)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
os.getcwd()

'/mnt/d/works/paperworks/2023/aging/Classifiers/benchmark-food'

In [7]:
model_path = "distilBERT-ko-wikipedia-classifier.prm"
model.load_state_dict(torch.load(model_path))
model.to("cuda:0")  # Make sure to move the model to the desired device


DistilBertClassifier(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (li

In [8]:
test = pd.read_csv("data/food_test_data.csv")

In [9]:
test_list_names = test["식품오타"].values.tolist()
test_list_labels = test["label"].values.tolist()

In [10]:
words = test_list_names
labels = test_list_labels 

In [11]:
len(words)

584

In [12]:
len(labels)

584

In [13]:
MAX_LENGTH = 100  # Adjust as needed

def tokenize_data(texts, labels):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])

    return input_ids, attention_masks, labels

#texts = ["I love this product!", "This is terrible."]  # Sample texts
#labels = [1, 0]  # Corresponding labels

test_input_ids, test_attention_masks, test_labels = tokenize_data(words, labels)

In [14]:
from torch.utils.data import DataLoader, TensorDataset

# Convert data to tensors
input_ids = torch.tensor(test_input_ids)
attention_masks = torch.tensor(test_attention_masks)
labels = torch.tensor(test_labels)

dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, batch_size=16)  # Adjust batch size as needed


In [15]:
# inference time check
model.eval()
all_predictions = []
all_true_labels = []

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# start 
start.record()

with torch.no_grad():
    for batch in dataloader:
        batch_input_ids, batch_attention_masks, batch_labels = [b.to("cuda:0") for b in batch]
        
        logits = model(batch_input_ids, batch_attention_masks)
        _, predictions = torch.max(logits, dim=1)
        
        all_predictions.extend(predictions.cpu().numpy())
        all_true_labels.extend(batch_labels.cpu().numpy())


end.record()

# wait until the event is done
torch.cuda.synchronize()

# check
elapsed_time = start.elapsed_time(end)  # msec 
execution_per_sample = (elapsed_time / 1000) / len(labels)
print(f"Elapsed time: {execution_per_sample} seconds")  # convert it to sec



Elapsed time: 0.49995648193359377 seconds


In [16]:
correct_predictions = sum(p == t for p, t in zip(all_predictions, all_true_labels))
accuracy = correct_predictions / len(all_true_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6507


In [18]:
from sklearn.metrics import recall_score, precision_score, f1_score

In [52]:
recall = recall_score(all_true_labels, all_predictions, average='micro')
recall

0.6506849315068494

In [53]:
precision_score(all_true_labels, all_predictions, average='micro')

0.6506849315068494

In [55]:
f1_score(all_true_labels, all_predictions, average='micro')

0.6506849315068494